In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
from nsetools import Nse

In [2]:
nse = Nse()
stocks = nse.get_stock_codes()

In [3]:
my_columns = ['Name','Previous_Closing_Price','Opening_Price','Closing_Price','Day_High_Price','Day_low_Price','Average_Price','Current_Price','Shares_To_Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

In [4]:
names = []
top_50 = nse.get_preopen_nifty()
for i in range(len(top_50)):
    names.append(top_50[i]['symbol'])

In [5]:
for stock in names:
        Data = nse.get_quote(stock)
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            Data['symbol'],
            Data['previousClose'],
            Data['open'],
            Data['closePrice'],
            Data['dayHigh'],
            Data['dayLow'],
            Data['averagePrice'],
            Data['lastPrice'],
            'N/A',

        ],
            index = my_columns
        ),
            ignore_index = True
        )


In [6]:
#Calculating the Number of shares to buy
amount = input("Enter the amount you want to spend: ")

#Making sure the value is integer.
try:
    val = float(amount)
except ValueError:
    print("That's not a number!\n Please try again: ")
    amount = input("Enter the amount: ")
    val = float(amount)

#To invest
Size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Shares_To_Buy'] = math.floor(Size/final_dataframe.loc[i, 'Current_Price'])



In [7]:
writer = pd.ExcelWriter('Recommanded stocks.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended stocks', index=False)

In [8]:
background_color = '#FFFFFF'
font_color ='#000000'

string_format = writer.book.add_format(
    {
    'font_color':font_color,
    'bg_color':background_color,
    'border': 1
    }
)

Rupee_format = writer.book.add_format(
    {
    'num_format':'0',
    'font_color':font_color,
    'bg_color':background_color,
    'border': 4
    }
)


In [9]:
column__formats ={
    'A':['Name', string_format],
    'B':['Previous_Closing_Price',Rupee_format],
    'C':['Opening_Price',Rupee_format],
    'D':['Closing_Price',Rupee_format],
    'E':['Day_High_Price',Rupee_format],
    'F':['Day_low_Price',Rupee_format],
    'G':['Average_Price',Rupee_format],
    'H':['Current_Price',Rupee_format],
    'I':['Shares_To_Buy',Rupee_format]
}
for c in column__formats.keys():
    writer.sheets['Recommended stocks'].set_column(f"{c}:{c}", 18, column__formats[c][1])
    writer.sheets['Recommended stocks'].write(f'{c}1',column__formats[c][0],column__formats[c][1])

In [10]:
writer.save()